
$\qquad$ $\qquad$ $\qquad$  **TDA206/DIT206 Discrete Optimization: Assignment 1 -- Modelling and Solving LPs** <br />
$\qquad$ $\qquad$ $\qquad$                   **Grader: N/A** <br />
$\qquad$ $\qquad$ $\qquad$                     **Due Date: 27th January** <br />
$\qquad$ $\qquad$ $\qquad$                   **Submitted by: Olof Lindberg, 020725-0192, ololin0725@gmail.com** <br />
$\qquad$ $\qquad$ $\qquad$                   **Submitted by: Rikard Roos, 941003-9292, rikardro@student.chalmers.se** <br />


---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook, and equations wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook do not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your Jupyter notebook run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Problem 1.

Consider the following LP problem:


\begin{aligned}
\mathcal{max} \quad 4x_1-2x_2+5x_3+6x_4+7x_5\\
\textrm{s.t} \\
2x_1 + 2x_2 - 4x_3 + 4x_4 + 8x_5 \leq 6\\
2x_1 + x_2 - 2x_3 - x_4 - 3x_5 \geq -1\\
5x_1 - 2x_2 + 4x_3 + 4x_4 + 2x_5 = 5\\
2x_1 - 2x_2 + 5x_3 + 3x_4 + x_5 \leq 4\\
\vec x \geq \vec 0
\end{aligned}

* Use CVXPY to solve the LP above. Submit your code, and print the solution vector and objective value.

In [341]:
import cvxpy as cp

# Variables
x_1 = cp.Variable()
x_2 = cp.Variable()
x_3 = cp.Variable()
x_4 = cp.Variable()
x_5 = cp.Variable()

# Objective
obj = cp.Maximize( 4 * x_1 - 2 * x_2 + 5 * x_3 + 6 * x_4 + 7 * x_5)

# Constraints
constraints = [2 * x_1 + 2 * x_2 - 4 * x_3 + 4 * x_4 + 8 * x_5 <= 6, 
               2 * x_1 + x_2 - 2 * x_3 - x_4 - 3 * x_5 >= -1,
               5 * x_1 - 2 * x_2 + 4 * x_3 + 4 * x_4 + 2 * x_5 == 5,
               2 * x_1 - 2 * x_2 + 5 * x_3 + 3 * x_4 + x_5 <= 4,
               x_1 >= 0, x_2 >= 0, x_3 >= 0, x_4 >= 0, x_5 >= 0]

prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {opt}")
print(f"Solution vector: {[round(float(v.value), 3) for v in [x_1, x_2, x_3, x_4, x_5]]}")


Objective value: 9.220338951278354
Solution vector: [0.627, 2.814, 1.542, 0.0, 0.661]


# Problem 2.

There are 3 people who all start at the same location, and want to get to same end destination a total of $d=100$ units away. They have 1 bicycle, which they have to share. The amount of distance per unit time that the three people can cover walking and cycling is given below:

\begin{array}{l|c|c|} 
      & Walking Speed & Cycling Speed\\ \hline
 Person\ 1 &  1 & 6 \\
 Person\ 2 &  2 & 8 \\
 Person\ 3 &  1 & 6\\ \hline
\end{array}

Our goal is to minimize the time $t$ required for all people to reach the end destination. For each person consider 4 variables $x_i$, $u_i$, $y_i$, $z_i$, which respectively are the amount of time walking forwards, the amount of time walking backwards, the amount of time cycling forwards, and the amount of time cycling backwards. 

1. Formulate an LP to solve this problem for the generic where there are $n$ people trying to move a distance $d$, each with walking rate $w_i$ and biking rate $b_i$ for $i = 1\ldots n$.

    You should consider the following constraints
    - each person's total time for moving (forwards, backwards, walking, bike) not exceed the total time $t$
    - each person must make it all the way to $d$, recall that distance = velocity * time
    - the bike, riding forwards or backwards, can only be used for at most $t$ units of time.
    - the bike's distance to the start is at most $d$, remember that it can move both forwards and backwards.
    - time cannot be negative
   
2. Implement the generic LP in CVXPY and solve for the specific case of 3 people given above. Submit your code and write down objective and solution. Also confirm that the optimal solution satisfies $u_i = 0$ (i.e. no one ever walks backwards) and $y_iz_i = 0$ for all persons (i.e. no person ever cycles both forwards and backwards).

3. Use CVXPY to show what would happen if there was a 4th person who could walk at rate $3$ and cycle at rate $9$.


In [342]:
import cvxpy as cp

# Generic solution for n people to walk distance d
# n[i] = (w_i, b_i) 

def minimize_time(rates, d):
    # Initialize variables
    people = []

    for _ in range(len(rates)):
        # Each person has positive time for each variable
        people.append([cp.Variable(nonneg=True) for _ in range(4)])

    t = cp.Variable()

    # Objective function
    obj = cp.Minimize(t)

    # Constraints
    constraints = []

    # Each person's total time for moving cannot exceed the total time t
    for p in people:
        constraints.append(sum(p) <= t)

    # Each person must make it all the way to d
    for i, (w, b) in enumerate(rates):
        p = people[i]
        constraints.append(w * (p[0] - p[1]) + b * (p[2] - p[3]) >= d)

    # The bike can only be used for at most t units of time
    total_bike_time = sum(p[2] + p[3] for p in people)
    constraints.append(total_bike_time <= t)

    # The bike's distance from the start cannot exceed d
    total_bike_distance = sum(rates[i][1] * (people[i][2] - people[i][3]) for i in range(len(people)))
    constraints.append(total_bike_distance <= d)

    # Time cannot be negative
    constraints.append(t >= 0)

    # Solution
    prob = cp.Problem(obj, constraints)
    opt = prob.solve()

    print(f"Objective value: {round(opt, 3)}")
    for i, p in enumerate(people):
        print(f"Person {i + 1}: {[round(float(v.value), 3) for v in p]}")


In [343]:
# Implement the generic LP in CVXPY and solve for the specific case of 3 people given above.
    
n = [(1, 6), (2, 8), (1, 6)] 
minimize_time(n, 100)


Objective value: 55.0
Person 1: [46.0, 0.0, 9.0, 0.0]
Person 2: [54.0, 0.0, 0.0, 1.0]
Person 3: [46.0, 0.0, 9.0, 0.0]


In [344]:
# 3. Use CVXPY to show what would happen if there was a 4th person who could walk at rate $3$ and cycle at rate $9$.

n = [(1, 6), (2, 8), (1, 6), (3, 9)]
minimize_time(n, 100)


Objective value: 47.608
Person 1: [37.13, 0.0, 10.478, 0.0]
Person 2: [46.811, 0.0, 0.797, 0.0]
Person 3: [37.13, 0.0, 10.478, 0.0]
Person 4: [44.039, 0.0, 0.0, 3.569]


# Question 3.
Use CVXPY to model and solve the other examples in the textbook (sections 2.1 -- 2.6).

In [345]:
# Optimized diet
import cvxpy as cp 

# Variables
x1 = cp.Variable()
x2 = cp.Variable() 
x3 = cp.Variable()

# Objective function
obj = cp.Minimize(0.75 * x1 + 0.5 * x2 + 0.15 * x3)

# Constraints
constraints = [35 * x1 + 0.5 * x2 + 0.5 * x3 >= 0.5,
              60 * x1 + 300 * x2 + 10 * x3 >= 15,
              30 * x1 + 20 * x2 + 10 * x3 >= 4,
              x1 >= 0, x2 >= 0, x3 >= 0]

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
vec = [x1.value, x2.value, x3.value]

print(f"Objective value: {opt:.2g}")
print(f"Solution vector: {['{:g}'.format(float('{:.2g}'.format(i * 1000))) for i in vec]}")



Objective value: 0.071
Solution vector: ['9.5', '38', '290']


In [346]:
# Flow in a network

# Variables
x = cp.Variable(10)

# Objective function
obj = cp.Maximize(x[0] + x[1] + x[2])

# Constraints 
constraints = [
    x[0] <= 3, -3 <= x[0],
    x[1] <= 1, -1 <= x[1],
    x[2] <= 1, -1 <= x[2],
    x[3] <= 1, -1 <= x[3],
    x[4] <= 1, -1 <= x[4],
    x[5] <= 3, -3 <= x[5],
    x[6] <= 4, -4 <= x[6],
    x[7] <= 4, -4 <= x[7],
    x[8] <= 4, -4 <= x[8],
    x[9] <= 1, -1 <= x[9],
    x[0] == x[3] + x[4],
    x[1] + x[3] == x[5],
    x[2] == x[6] + x[7],
    x[4] + x[6] == x[8],
    x[5] + x[7] == x[9]
]

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()

print(f"Objective value: {opt:.2g}")
print(f"Optimal solution: {[round(float(v.value), 3) for v in x]}")



Objective value: 4
Optimal solution: [2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.193, -1.193, 3.193, 0.807]


In [347]:
# Ice cream all year around

# Variables
x = [0] + [cp.Variable(nonneg=True) for _ in range(12)]
s = [0] + [cp.Variable(nonneg=True) for _ in range(11)] + [0]
y = [cp.Variable(nonneg=True) for _ in range(12)]
z = [cp.Variable(nonneg=True) for _ in range(12)]
# Approximation of values from the graph
d = [350, 340, 430, 640, 640, 540, 700, 680, 350, 420, 400, 600]

# Objective function
obj = cp.Minimize(50 * sum(y) + 50 * sum(z) + 20 * sum(s))

# Constraints
constraints = []

for i in range(12):
    constraints.append(x[i+1] + s[i] - s[i+1] == d[i])

for i in range(12):
    constraints.append(x[i+1] - x[i] == y[i] - z[i])

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {round(opt)}")


Objective value: 52175


In [348]:
# Fitting a line

# Variables
x = [17, 27, 30, 34, 38, 38, 43, 46, 55, 55]
y = [51, 48, 45, 45, 44, 58, 45, 42, 42, 39]
e = cp.Variable(10)
a = cp.Variable()
b = cp.Variable()

# Objective function
obj = cp.Minimize(sum(e))

# Constraints 
constraints = []
for i in range(10):
    constraints.append(e[i] >= a * x[i] + b - y[i])
    constraints.append(e[i] >= -(a * x[i] + b - y[i]))

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {round(opt)}")
print(f"The line: y = {a.value}x + {b.value}")

Objective value: 22
The line: y = -0.3103448276771387x + 56.27586207076257


In [349]:
# Separation of points

# Approximation of values
p = [(15, 16), (10, 20), (9, 30), (23, 24), (30, 31), (35, 30), (37, 23), (44, 48)]
q = [(15, 11), (25, 15), (26, 18), (30, 16), (36, 13), (38, 22)]


# Variables
a = cp.Variable()
b = cp.Variable()
delta = cp.Variable()

# Objective function
obj = cp.Maximize(delta)

# Constraints
constraints = []
for i in range(len(p)):
    constraints.append(p[i][1] >= a * p[i][0] + b + delta)

for j in range(len(q)):
    constraints.append(q[j][1] <= a * q[j][0] + b - delta)

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {round(opt)}")
print(f"The line: y = {round(float(a.value))}x + {round(float(b.value))}")

Objective value: 1
The line: y = 0x + 10


In [350]:
# Largest disk in a convex polygon
import math

# Random values
a = [1, 1, -1, -1, 0.5, 0.5, -0.5, -0.5]
b = [1, -1, 1, -1, 1.5, -1.5, 1.5, -1.5]
k = 4

# Variables
r = cp.Variable()
s1 = cp.Variable()
s2 = cp.Variable()

# Objective function
obj = cp.Maximize(r)

# Constraints 
constraints = []
for i in range(k):
    constraints.append((s2 - a[i] * s1 - b[i]) / math.sqrt (a[i] ** 2 + 1) >= r)
    
for j in range(len(a)-k):
    constraints.append((s2 - a[k+j] * s1 - b[k+j]) / math.sqrt (a[k+j] ** 2 + 1) <= -r)

# Solution
prob = cp.Problem(obj, constraints)
opt = prob.solve()
print(f"Objective value: {opt}")

Objective value: -0.9872652454539556
